In [2]:
class Pessoa:
    def __init__(self, nome:str, idade: int, peso: float) -> None:
        self.nome = nome 
        self.idade = idade 
        self.peso = peso

In [3]:
lucas = Pessoa("Lucas de Sousa", 31, 101)

In [5]:
lucas
lucas.nome

'Lucas de Sousa'

In [6]:
from pydantic import BaseModel


class pydPessoa(BaseModel):
    nome: str
    idade: int
    peso: float

In [8]:
lucas2 = pydPessoa(nome="Lucas de sousa", idade=21, peso=101.2)
lucas2

pydPessoa(nome='Lucas de sousa', idade=21, peso=101.2)

In [10]:
from langchain.pydantic_v1 import BaseModel, Field 
from typing import Optional 
from enum import Enum 

class UnidadeEnum(str, Enum):
    celsius = 'celsius'
    fahrenheit = 'fahrenheit'
    
class ObterTemperaturaAtual(BaseModel):
    """Obtém a temperatura atual de uma determinada localidade"""
    local:str = Field(description="O nome da cidade", examples=["São Paulo", "Porto Alegre"])
    unidade:Optional[UnidadeEnum]

In [11]:
from langchain_core.utils.function_calling import convert_to_openai_function 

tool_temperature = convert_to_openai_function(ObterTemperaturaAtual)
tool_temperature

{'name': 'ObterTemperaturaAtual',
 'description': 'Obtém a temperatura atual de uma determinada localidade',
 'parameters': {'type': 'object',
  'properties': {'local': {'description': 'O nome da cidade',
    'examples': ['São Paulo', 'Porto Alegre'],
    'type': 'string'},
   'unidade': {'description': 'An enumeration.',
    'enum': ['celsius', 'fahrenheit'],
    'type': 'string'}},
  'required': ['local']}}

In [12]:
from langchain_openai import ChatOpenAI 

chat = ChatOpenAI() 

res = chat.invoke("Qual é a temperatura de Porto Alegre", functions=[tool_temperature])
res

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"Porto Alegre"}', 'name': 'ObterTemperaturaAtual'}}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 105, 'total_tokens': 128}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-42010111-144d-4078-9c46-fe7782dcf63f-0', usage_metadata={'input_tokens': 105, 'output_tokens': 23, 'total_tokens': 128})

In [13]:
res = chat.invoke("Olá", functions=[tool_temperature], function_call={"name": "ObterTemperaturaAtual"})

In [14]:
res

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"São Paulo"}', 'name': 'ObterTemperaturaAtual'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 112, 'total_tokens': 119}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-440b22b7-300f-4f89-b6e5-bac11b64997f-0', usage_metadata={'input_tokens': 112, 'output_tokens': 7, 'total_tokens': 119})

In [16]:
from langchain.prompts import ChatPromptTemplate 

prompt = ChatPromptTemplate.from_messages([
	("system", "Você é um assistente amigável chamado Issac"),
	("user", "{input}")
])

chain = prompt | chat.bind(functions=[tool_temperature])

In [17]:
chain.invoke({"input": "Olá"})

AIMessage(content='Olá! Como posso te ajudar hoje?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 111, 'total_tokens': 123}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-39bdd676-8d1b-495e-8d27-49490971c989-0', usage_metadata={'input_tokens': 111, 'output_tokens': 12, 'total_tokens': 123})

In [18]:
chain.invoke({"input": "Qual a temperatura de Minas Gerais?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"Minas Gerais"}', 'name': 'ObterTemperaturaAtual'}}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 118, 'total_tokens': 140}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-6db49ec6-05de-4074-8a0e-e3d126d78032-0', usage_metadata={'input_tokens': 118, 'output_tokens': 22, 'total_tokens': 140})